In [59]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [60]:
# Load the Excel file
file_name = "SES_2024_tidy.xlsx.coredownload.xlsx"
sheet_name = "T3.5"

# Read the data from the Excel file
df = pd.read_excel(file_name, sheet_name=sheet_name)
df = df.fillna(0)

# Treat the random 's' as 0
df['Kwh_Per_Acc_Cleaned'] = df['Kwh Per Acc'].apply(lambda x: 0 if isinstance(x, str) and 's' in x else x) 

# Filter the data for the year 2024 and 'Dwelling Type' as 'overall'
filtered_data = df[(df['Year'] == 2024) & (df['Dwelling Type'] == 'Overall')]

# Calculate total consumption per region
regions = ['Central Region', 'East Region', 'North East Region', 'North Region', 'West Region']
demand_kW_per_region = {
    region: filtered_data[filtered_data['Region'] == region]['Kwh_Per_Acc_Cleaned'].astype(float).sum()
    for region in regions
}

demand_kW_per_region

{'Central Region': 88877.8,
 'East Region': 27358.199999999997,
 'North East Region': 23995.5,
 'North Region': 23482.0,
 'West Region': 19945.699999999997}

In [61]:
df_carpark = pd.read_csv('Checkmark2.csv')

# Calculate the number of car parks per region
carpark_per_region = {
    region: df_carpark[df_carpark['region'] == region].shape[0]
    for region in regions
}

# Output the result
carpark_per_region

{'Central Region': 541,
 'East Region': 400,
 'North East Region': 493,
 'North Region': 303,
 'West Region': 506}

In [62]:
# Define other parameters
types_of_panels = ['Monocrystalline', 'Polycrystalline', 'Thin-film']
efficiency = {'Monocrystalline': 0.15, 'Polycrystalline': 0.13, 'Thin-film': 0.07}
var_cost = {'Monocrystalline': 266.125, 'Polycrystalline': 265.85, 'Thin-film': 265.5}
fixed_cost = 39457.75
available_space_m2 = 2559.5
budget_limit = 1_000_000_000  # Budget limit in dollars

# Calculate the total area by region constraint
area_by_region_constraint = {
    region: carpark_per_region[region] * available_space_m2
    for region in carpark_per_region
}

area_by_region_constraint

{'Central Region': 1384689.5,
 'East Region': 1023800.0,
 'North East Region': 1261833.5,
 'North Region': 775528.5,
 'West Region': 1295107.0}

In [63]:
# Create a new model
model = gp.Model("solar_optimization")

# Decision variables
x = model.addVars(regions, types_of_panels, vtype=GRB.INTEGER, name="x")
y = model.addVars(regions, vtype=GRB.BINARY, name="y")

In [64]:
model.setObjective(
    gp.quicksum(fixed_cost * y[region] + gp.quicksum(var_cost[panel] * x[region, panel] for panel in types_of_panels) for region in regions),
    GRB.MINIMIZE
)

<h3>Can budget by maximum number of panels you want to build if you do this way it's 1000 in the last line</h3>

In [65]:
# # Constraints
# for i in regions:
#     # Demand constraint with maximum power per panel
#     model.addConstr(
#         gp.quicksum(efficiency[j] * max_power_per_panel_kW * x[i, j] for j in types_of_panels) >= demand_kW_per_region[i],
#         name=f"demand_{i}"
#     )

#     # Space constraint
#     model.addConstr(
#         gp.quicksum(x[i, j] * panel_area_m2 for j in types_of_panels) <= available_space_m2,
#         name=f"space_{i}"
#     )

#     # Installation constraints
#     for j in types_of_panels:
#         model.addConstr(x[i, j] <= 1000 * y[i, j], name=f"install_{i}_{j}")

<h3>Or if you do this way then you budget by the money set in the cell above. For now I put $1 mil</h3>

In [66]:
# Add constraints
for region in regions:
    model.addConstr(gp.quicksum(efficiency[panel] * x[region, panel] for panel in types_of_panels) >= demand_kW_per_region[region], f"Demand_Constraint_{region}")
    model.addConstr(gp.quicksum(10*x[region, panel] for panel in types_of_panels) <= area_by_region_constraint[region], f"Space_Constraint_{region}")
    model.addConstr(fixed_cost * y[region] + gp.quicksum(var_cost[panel] * x[region, panel] for panel in types_of_panels) <= budget_limit, f"Budget_Constraint_{region}")
    # for panel in types_of_panels:
    #     model.addConstr(x[region, panel] <= available_space_m2 * y[region], f"Big_M_Constraint_{region}_{panel}")

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    for region in regions:
        print(f"y[{region}] = {y[region].X}")
        for panel in types_of_panels:
            print(f"x[{region},{panel}] = {x[region,panel].X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 15 rows, 20 columns and 50 nonzeros
Model fingerprint: 0xb6e1d799
Variable types: 0 continuous, 20 integer (5 binary)
Coefficient statistics:
  Matrix range     [7e-02, 4e+04]
  Objective range  [3e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 1e+09]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


In [67]:
# Set the txt file to see the logs
model.setParam('LogFile', 'gurobi_log.txt')
model.setParam('OutputFlag', 1) 

# Optimize the model
model.optimize()

# In case we mess up and we wanna see the variables 
if model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")

Set parameter LogFile to value "gurobi_log.txt"
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 15 rows, 20 columns and 50 nonzeros
Model fingerprint: 0xb6e1d799
Variable types: 0 continuous, 20 integer (5 binary)
Coefficient statistics:
  Matrix range     [7e-02, 4e+04]
  Objective range  [3e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 1e+09]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction se

In [68]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in regions:
        for j in types_of_panels:
            print(f"Region: {i}, {j}: {x[i, j].x} panels, Build: {y[i, j].x}")
    print(f"Total Cost: ${model.objVal}")
else:
    print("No optimal solution found.")

No optimal solution found.


In [69]:
for constr in model.getConstrs():
    print(f"Constraint: {constr.ConstrName}, Expression: {model.getRow(constr)}, RHS: {constr.RHS}")

Constraint: Demand_Constraint_Central Region, Expression: 0.15 x[Central Region,Monocrystalline] + 0.13 x[Central Region,Polycrystalline] + 0.07 x[Central Region,Thin-film], RHS: 88877.8
Constraint: Space_Constraint_Central Region, Expression: 10.0 x[Central Region,Monocrystalline] + 10.0 x[Central Region,Polycrystalline] + 10.0 x[Central Region,Thin-film], RHS: 1384689.5
Constraint: Budget_Constraint_Central Region, Expression: 266.125 x[Central Region,Monocrystalline] + 265.85 x[Central Region,Polycrystalline] + 265.5 x[Central Region,Thin-film] + 39457.75 y[Central Region], RHS: 1000000000.0
Constraint: Demand_Constraint_East Region, Expression: 0.15 x[East Region,Monocrystalline] + 0.13 x[East Region,Polycrystalline] + 0.07 x[East Region,Thin-film], RHS: 27358.199999999997
Constraint: Space_Constraint_East Region, Expression: 10.0 x[East Region,Monocrystalline] + 10.0 x[East Region,Polycrystalline] + 10.0 x[East Region,Thin-film], RHS: 1023800.0
Constraint: Budget_Constraint_East R